This notebook describes the steps for the White nationalist rhetoric example (first application). Data is from Siegel et al 2021. Prepared by Annamaria Prati and Yehu Chen.

# Setup

Let's set ourselves up. First, load the required libraries. Note that `torch` should be version 2.6.0 and `gpytorch` should be version 1.8.1. 

Next, set a seed so that the results are consistent each time. We will also set the default data type to be `float64`, which retains two decimal points (helps the matrix operations) and helps prevent data type mismatches. 

Finally, load the dataset. We have provided the dataset in a subfolder for this example on the Github repository.

In [11]:
# load required libraries
import torch
import numpy as np
import pandas as pd
import gpytorch
from scipy.stats import norm
from matplotlib import pyplot as plt
from datetime import datetime
from gpytorch.means import LinearMean
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.likelihoods import GaussianLikelihood

print(torch.__version__)
print(gpytorch.__version__)

# set a random seed so results are consistent each time you run the code
torch.manual_seed(12345)

# set default data type to float64 for accuracy
torch.set_default_dtype(torch.float64)

# load the dataset with the daily proportion of white nationalist tweets
# this file should be in a folder called 'data' with the name 'white_nationalist_random.csv'
data = pd.read_csv("./data/white_nationalist_random.csv")

2.6.0
1.8.1


# Interrupted Time Series Framework

Let's orient ourselves mathematically. 

Let $y_t$ be the proportion of white nationalist tweets at time $t$, $T$ indicate time, and  $D_t$ be a dummy variable indicating whether it is before ($D_t=0) $ or after ($D_t=1$) Election Day. The main model of interest is:

$$y_t=\beta_0 + \beta_1(T) + \beta_2(D_t) + \beta_3 (D_tT).$$

In the interrupted time series analysis framework, this model allows researchers to test the claim of a persistent change in speech after Election Day -- both in terms of level ($\beta_2$) and slope ($\beta_3$)

# Data Preparation

Now we need to prepare our data for the model. First, we will change the date so that it is an integer representing the number of time periods that have passed since the start of the dataset. We do this since Gaussian processes work best when inputs are numeric. 

Next, we will rescale the values for the outcome variable $y$ so that they have better numerical performance. 

Third, we will create our input features: time, when "treatment" starts (i.e. the election happened), and the interaction between the two. 

Finally, we will find the time index for election day. This will faciliate our examination of what happens before or after treatment later.

In [12]:
# convert date column to number of days since the start of the dataset
xs = data.date
xs = xs.apply(lambda x: (datetime.strptime(x,"%Y-%m-%d")-datetime.strptime(xs[0],"%Y-%m-%d")).days)

# rescale the values for better numerical performance
y_scale = 1e6
y = torch.tensor(data.norm_hate.values).double() * y_scale

# create the input features: time, election indicator, and their interaction
x = torch.tensor(np.array([xs.values, data.election.values, xs.values*data.election.values]).T).double()

# find the index of election day
election_day_index = np.where(x[:,1])[0][0]

# Why Gaussian Process Regression?

Instead of focusing on regression coefficients ($\beta_0, \beta_1, \beta_2, \beta_3$), Gaussian Process Regression allows us to estimate the function $f(t)$ modeling tweet proportions directly by constructing an appropriate model and prior as well as calculating the full posterior.

This shifts the emphasis from parametric inference toward the substantive question of interest: Did the frequency of white nationalist tweets increase after Election Day?

Formally, we estimate the post-election effect as the average difference between the GPR prediction for observed post-election outcomes and the counterfactual prediction of what would have been observed had there been no treatment effect, controlling for pre-election trends.

That is,

$$
\text{Effect} = \frac{\sum_{t>E}\big(f_{t}|D_{t}=1, \mathbf{X}_{-t} -  f^\ast_i | D_{t}=0, \mathbf{X}_{-t} \big)}{\sum_t I(t>E)} 
$$
where $E$ indicates Election Day and $I(\cdot)$ is the usual indicator function. This is the averaged difference between our GPR for the predicted value of $f_{t}$ and the predicted value we would have observed for that day factoring out any linear ``bump'' $(f_t^\ast)$ for the post-election period. This approach directly recovers the average treatment effect of Election Day on speech, while avoiding strict linearity assumptions.

# Model Specification

The general math "behind the scenes" is as follows. As explained in detail in the paper, a Gaussian Process is defining a distribution over functions, meaning we assume 
$$
f(x) \sim \mathcal{GP}(m(x), k(x, x'))
$$

In this particular model, the mean function is linear to capture the overall trend in the data:  
  $$
  m(x) = \beta_0 + \beta_1 x
  $$
Based on the way we set up our input features, we know that our specific mean function is:
    $$
    \mu(T, D_t) = \beta_0 + \beta_1 T + \beta_2 D_t + \beta_3 D_t T
    $$

- $T$ is time (in days)
- $D_t$ is an indicator for whether it's election day or after (a binary variable)
- this form allows for:
    - a linear time trend ($\beta_1 T$)
    - a level shift on election day ($\beta_2 D_t$)
    - and an interaction ($\beta_3 D_t T$), allowing the trend to change after the election


And the covariance function (or kernel) is the RBF (also called the Squared Exponential, or SE) kernel:
  $$
  k(x, x') = \sigma^2 \exp\left( -\frac{(x - x')^2}{2\ell^2} \right)
  $$

Recall that this controls how much two observations are expected to move together based on how close they are in time (as specified by the `active_dims=[0]` portion of our code).  

  $\ell$ is the lengthscale — it determines how fast correlation decays with distance.  
  $\sigma^2$ is the outputscale — it controls the overall variation in the function.

Together, this GPR prior models the function $f(x)$ as having a linear trend and smooth deviations from that trend.

# Constructing and Initializing the `GPModel` Class in `gpytorch`

To operationalize this in `gpytorch`, we begin by defining our customized Gaussian process model as a class. It inherits properties from the off-the-shelf `ExactGP` model (from `gpytorch`), which is appropriate when we have a full (or randomly sampled) dataset and can compute exact inference (as opposed to approximation methods). 

After initilization, we define the mean function and kernel function. We set the mean function to be linear, which assumes that this is the baseline trend. The GP will be modeling deviations from this baseline trend. The `input_size` here is dynamically defined but we know it is equal to 3: time, the election indicator, and the interaction. For the kernel/covariance function, we use the RBF kernel, which assumes smooth changes. This is modeling how outputs relate to each other over time. By specifying `active_dims[0]` we have instructed the covariance function to only use the first input column, which is our time since election data. Note that this is NOT based on the election status at all.

In the second part of the class definition, we instruct how the model should make predictions for any input. The instructions here say to compute the mean at each input using the linear trend we defined earlier; and to compute the covariance using the kernel and time column as we defined earlier. The model then returns the predictions as a multivariate normal distribution. 

In [13]:
# define a gaussian process model with a linear mean and RBF (smooth) kernel
class GPModel(gpytorch.models.ExactGP):
    # initialize our method: takes training data and the likelihood model
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        # linear mean function: a straight line trend
        self.mean_module = LinearMean(input_size=train_x.shape[1], bias=True)
        # covariance function: RBF kernel for capturing smooth patterns over time
        self.covar_module = ScaleKernel(RBFKernel(active_dims=[0]))

    # define what the model does when making predictions
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

Now that we have defined our custom Gaussian process model class (`GPModel`), we assign the likelihood to be Gaussian (which assumes normally distributed noise), and then we create an instance of the model (called `model`) using our inputs $x$, outputs $y$, and the likelihood.

This corresponds to the the model:

$$
y_t \sim \mathcal{N}(f(x_t), \sigma_{\text{noise}}^2)
$$

where $f(x_t)$ is drawn from a GP prior defined by the mean and kernel functions.

In [14]:
# initialize the model
likelihood = GaussianLikelihood()
model = GPModel(x, y, likelihood).double()

Now we set the initial values for the model's hyperparameters in a dictionary. At a high level, these are initial "guesses" for a hyperparameter. The model will update these during training based on the data (except for the lengthscale, more on that soon). 

We set three `mean_module.weights` since we have three input features: time, election indicator, and the interaction. By setting the initial values to zero, the model starts by assuming no trend or treatment effect -- this will be learned from the data.

The `covar_module.outputscale` is the output scale, controlling overall variability of the Gaussian process. 1 is a "medium-sized" guess.

The `covar_module.base_kernel.lengthscale` is the lengthscale, controlling the smoothness of the Gaussian process. It is set to `torch.tensor([90.])` to reflect that we believe opinions within a 90-day window should be similar. This is because we believe that hate speech trends in general should be slow to change. This value is later frozen for the test of whether hate speech has a "jump" after the election.

Finally, `likelihood.noise` is a noise parameter that is the amount of random noise in the observations. 

With all the initial values for the hyperparameters set, we call the model’s `.initialize()` method and pass the values using `**` to unpack the dictionary.

In [15]:
# set initial values for model parameters
hypers = {
    'mean_module.weights': torch.tensor([0,0,0]),
    'covar_module.outputscale': 1,
    'covar_module.base_kernel.lengthscale': torch.tensor([90.]),
    'likelihood.noise': 1,
}    
model = model.initialize(**hypers)

# Training the Model

We switch the model and likelihood into "training" mode. In `gpytorch`, this enables parameter updates during optimization. 

At this point we will also freeze the lengthscale value so that it is not updated, but we will optimize the other model parameters using the Adam optimization algorithm (a popular choice) with a learning rate of 0.05. Higher values for the learning rate mean larger steps. This is faster, but potentially less stable. Lower values for the learning rate mean smaller steps, which is slower. Note that we have wrapped the model's parameters as a `set` to de-duplicate any parameters -- this is technically not necessary but a safer option given our customization might become more complicated.

Finally we define the loss function to marginal log likelihood for exact Gaussian process models. We will need this later for assessing the quality of our predictions.

Mathematically, we want to maximize the marginal log-likelihood of the data under the GP prior:

$$
\log p(\mathbf{y} \mid \mathbf{X}, \boldsymbol{\theta}, \boldsymbol{\beta}) = -\frac{1}{2} (\mathbf{y} - \mu(\mathbf{X}))^\top K_{\theta}^{-1} (\mathbf{y} - \mu(\mathbf{X})) - \frac{1}{2} \log |K_{\theta}| - \frac{n}{2} \log 2\pi
$$

- $K_{\theta}$ is the covariance matrix defined by the kernel and its hyperparameters $\boldsymbol{\theta}$.
- $\mu(\mathbf{X})$ is the mean vector defined by the linear mean function.
- $\boldsymbol{\beta}$ are the parameters of the mean function.


In [16]:
# train the model
model.train()
likelihood.train()

# freeze lengthscale so it doesn't get updated
model.covar_module.base_kernel.raw_lengthscale.requires_grad = False

# set up optimizer to update the model parameters
optimizer = torch.optim.Adam(set(model.parameters()), lr=0.05)

# define the loss function for gaussian processes
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

Now we train the model! We have set it for 500 iterations or learning steps (you'll probably want fewer while building out the model, but probably more for finalized results) and we will save our losses in a list in case we need it later. Note that this might take awhile, but we have set it to print an update for you ever 50 iterations.

`output` is calling the `forward` method in `GPModel` to compute the predictions for input $x$ (time, election indicator, and the interaction).

`loss` is calculating the negative marginal log likelihood between the model's predictions and our true values of $y$. A smaller loss is better. This is equivalent to maximizing the log likelihood (because `pytorch` default is to minimize). `loss.backward()` computes the gradients of the loss with respect to the model parameters, instructing the optimizer which direction to move the parameters to reduce loss. This is appended to a list.

Finally, `optimizer.step()` updates the model parameters using the gradients from `loss.backward()` and the learning rate (`lr=0.05`).


In [17]:
# run the training loop
training_iter = 500
losses = []
for i in range(training_iter):
    optimizer.zero_grad() # resets gradients before each training step
    output = model(x)
    loss = -mll(output, y)
    loss.backward()
    if i % 50 == 0:
        print('Iter %d/%d - Loss: %.3f '  % (i , training_iter, loss.item())) # loss from pytorch tensor
    losses.append(loss.item())
    optimizer.step()

Iter 0/500 - Loss: 2.873 
Iter 50/500 - Loss: 2.233 
Iter 100/500 - Loss: 2.146 
Iter 150/500 - Loss: 2.123 
Iter 200/500 - Loss: 2.114 
Iter 250/500 - Loss: 2.109 
Iter 300/500 - Loss: 2.107 
Iter 350/500 - Loss: 2.105 
Iter 400/500 - Loss: 2.104 
Iter 450/500 - Loss: 2.104 


# Evaluating the Model

Training is now done. We freeze our learned hyperparameters by switching the model and likelihood into "evaluation" mode for predictions. 

In [18]:
# switch model to evaluation mode for predictions
model.eval()
likelihood.eval()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

To obtain the posterior predictions for our observed data, we specify that we do not need to track gradients (`torch.no_grad()`, saves memory and computation time) and enable a setting that allows for faster computation of the prediction variance (`gpytorch.settings.fast_pred_var()`, at the cost of some precision). 

`out` is calling the `forward` method in `GPModel`, essentially calculating the posterior distribution for $x$ (time, the election indicator, and the interaction) and returning a multivariate normal distribution. 

`loss` is then calculating the negative marginal log likelihood of the predictions. Recall that `output` is defined when we trained the model. The `np.log(y_scale)` is to account for our previous scaling of the $y$ values (for stability purposes).  

`mu_f` is the mean of the posterior distribution. 

`lower` and `upper` are the lower and upper bounds of the confidence interval for the posterior predictions. 

Note that this step is necessary for us to calculate our performance statistics.

In [19]:
# get posterior predictions for observed data
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    out = model(x) # posterior distribution for input x
    loss = -mll(output, y)-np.log(y_scale)
    mu_f = out.mean.numpy()
    lower, upper = out.confidence_region()

C:\Users\miame\anaconda3\Lib\site-packages\gpytorch\models\exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


As discussed in the main text, as social scientists we are most likely not interested in the predictions but rather the difference between two scenarios: what if the election happened (observed) versus what if the election had not happened (counterfactual). We use our new prediction tool to model these two scenarios.  We do this by creating two copies of the inputs and changing the election indicator column such that one input is where the election occurred (i.e. the election indicator is 1) and one input is where the election did not occur (i.e. the election indicator is 0). 

Like before, we then get the posterior predictions for both sets of inputs.

In [20]:
# create counterfactual inputs: what if election had happened / not happened
# we create two sets of inputs with election set to 1 and 0
x_election_1 = x.clone().detach().requires_grad_(False)
x_election_1[:,1] = 1
x_election_0 = x.clone().detach().requires_grad_(False)
x_election_0[:,1] = 0

# get model predictions under both counterfactual scenarios
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    out1 = model(x_election_1)
    out0 = model(x_election_0)

We can now estimate the instantaneous effect of the election on election day:

In [21]:
# estimate the instantaneous effect of election day
effect = out1.mean.numpy()[election_day_index+1]-out0.mean.numpy()[election_day_index-1]
effect_std = np.sqrt((out1.variance.detach().numpy()[election_day_index+1] +
                      out0.variance.detach().numpy()[election_day_index-1]))
print("instaneous shift on election day: {:.2E} +- {:.2E}\n".format(effect/y_scale, effect_std/y_scale))

instaneous shift on election day: 7.35E-07 +- 2.79E-07



We can now also estimate the ATT:

In [22]:
# estimate the average treatment effect on the treated (post-election period only)
x_no_effect = x.clone().detach().requires_grad_(False)
x_no_effect[x[:,1]==1,1] = 0
x_no_effect[x[:,1]==1,2] = 0

with torch.no_grad():
    out0 = model(x_no_effect)
    mu_f0 = out0.mean.numpy()
    lower0, upper0 = out0.confidence_region()

mask1 = (x[:,0] >= election_day_index ) & (x[:,1]==1)
effect = out1.mean.numpy()[mask1].mean()-out0.mean.numpy()[mask1].mean()
effect_std = np.sqrt((out1.variance.detach().numpy()[mask1].mean() +
                      out0.variance.detach().numpy()[mask1].mean()))

print("ATT: {:.2E} +- {:.2E}\n".format(effect/y_scale, effect_std/y_scale))

ATT: 8.78E-07 +- 3.04E-07



Or we can calculate the difference between the observed values and the calculated counterfactual:

In [23]:
# calculate the difference between the factual and counterfactual predictions
diff = out.mean.numpy() - out0.mean.numpy()
print(diff)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

We can now examine model fit metrics:

In [24]:
# calculate model fit metrics
BIC = (3+2+2)*torch.log(torch.tensor(x.size(0))) + 2*loss*x.size()[0]
print(norm.cdf(-np.abs(effect/effect_std)))
print("log lik: {:4.4f} \n".format(-loss.numpy()*x.size(0)))
print("BIC: {:0.3f} \n".format(BIC))

0.001959731542149116
log lik: 8549.6996 

BIC: -17053.248 



Now we save our results into a dataframe and write to a csv.

In [25]:
# organize the prediction results into a dataframe
results = pd.DataFrame({"gpr_mean":mu_f/y_scale})
results['true_y'] = y/y_scale
results['gpr_lwr'] = lower/y_scale
results['gpr_upr'] = upper/y_scale
results['day'] = x[:,0].numpy().astype(int)
results["cf_mean"] = out0.mean.numpy() / y_scale

results.to_csv("output.csv", index=False)